<a href="https://colab.research.google.com/github/John-D-Boom/CompVision/blob/main/testing_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy
import matplotlib.pyplot as plt

In [2]:
torch

<module 'torch' from '/usr/local/lib/python3.8/dist-packages/torch/__init__.py'>

In [4]:
x = torch.ones(3,10)
print(x)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
